<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 32px;">Introduction</h1>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    This notebook presents and discusses how to encode the Weierstrass Elliptic Functions, namely:
    <ul>
        <li>$\wp(z) = \wp(z, g_2, g_3)$ - Weierstrass $\wp$ eliptic function for argument $z$ and characterised by lattice invariants $g_2$ and $g_3$ </li>
        <li>$\sigma(z) = \sigma(z, g_2, g_3)$ - Weierstrass $\sigma$ function for argument $z$ and characterised by lattice invariants $g_2$ and $g_3$ </li>
        <li>$\zeta(z) = \zeta(z, g_2, g_3)$ - Weierstrass $\zeta$ function for argument $z$ and characterised by lattice invariants $g_2$ and $g_3$ </li>
    </ul>

And the inverse function $\wp^{-1}$ of Wieierstrass. Additionally, definitions of helpful quantities such as:
    <ul>
        <li>$\omega_i = \omega_i (g_2, g_3)$ - half periods or lattice generators characterised by lattice invariants $g_2$ and $g_3$ </li>
    </ul>
    
The functions presented here are equivalent to what is currently available in Wolfram Mathematica 14 software in the field of Weierstrass Elliptic Functions.

The following two libraries are required for work:
- `mpmath`
- `numpy`

The code is based on definitions from the  <a href="https://dlmf.nist.gov/23" target="_blank">DLMF</a>  database
</p>
</div>


In [49]:
from mpmath import *
import numpy as np

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
We set two important parameters:

  `mp.dps = 25` : Specifies the precision of calculations to 25 decimal digits. By default, the precision is set to 15 digits, but you can adjust it to your needs.</li>

  `mp.pretty = True` : Enables pretty formatting of results. When this option is active, mpmath displays results in a more readable way, e.g. by rounding the numbers and using mathematical notation.
</ul>    
</div>    

In [2]:
mp.dps = 25
mp.pretty = True

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 32px;">$\wp$-function</h1>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">

Based on formulas <a href="https://dlmf.nist.gov/23.6" target="_blank">(23.6.1) and (23.6.5)</a> we can write
$$
    \wp (z) = \left( \frac{\pi}{2\omega_1} \theta_3(0,q)\theta_4(0,q) \frac{\theta_2(\pi z/(2\omega_1),q)}{\theta_1(\pi z/(2\omega_1),q)}  \right)^2 + \frac{\pi^2}{12 \omega_1^2} \left( \theta_2^4(0,q) + 2\theta_4^4(0,q)\right),
$$
where $\theta_i$ are Jacobi theta functions, $\omega_i$ are half-periods, $q=e^{i\pi\tau}$, $\tau=\omega_3/\omega_1$, and 
$$
     \operatorname{Im}(\tau)>0
$$

</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">Half-periods</h2>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
Based on formulas <a href="https://dlmf.nist.gov/23.6" target="_blank">(23.6.16-17)</a> for a Legendre elliptic integral of the first kind we can  have
$$
{K}^{2}=(K\left(k\right))^{2}=\omega_{1}^{2}(e_{1}-e_{3}),
$$
$$
{K}'^{2}=(K\left(k'\right))^{2}=\omega_{3}^{2}(e_{3}-e_{1}),
$$
where 
$$
    k^2 = \frac{e_2 - e_3}{e_1 - e_3}, \quad k'^2=\frac{e_1-e_2}{e_1-e_3}.
$$
So the half-periods are given by
$$
    \omega_1 = \pm \frac{1}{\sqrt{e_1-e_3}} K(k), \quad \omega_3 = \pm \frac{1}{\sqrt{e_3 - e_1}}K(k').
$$


The choice of the appropriate sign for half-periods nie ma nie jest dokładnie określony. Bierze się to wprost z ich definicji $\wp(z) = \wp(z + 2m \omega_1 + 2n\omega_3)$, gdzie $m,n\in N$. Ważne jest by zachodziło
$$
     \operatorname{Im}(\tau)>0.
$$
Więc wykożystujemu ten warunek w definicli $\omega_1$ przez co nie musimy się nim martwić w  $\omega_3$. By otrzymać $\omega_2$ wystarczy użyć relacji <a href="https://dlmf.nist.gov/23.2" target="_blank">(23.2.1)</a>:
$$
    \omega_1 + \omega_2 + \omega_3 = 0.
$$
</div>

In [56]:
def Omega1(g2, g3):
    # Współczynniki wielomianu 4*x**3 - g2*x - g3 = 0
    coeffs = [4, 0, -g2, -g3]

    # Znajdowanie pierwiastków numerycznych
    roots = np.roots(coeffs)
    
    # Sortowanie pierwiastków tak jak w SymPy
    roots = np.sort_complex(roots)
    e1, e2, e3 = roots

    # Obliczenie parametrów k^2 i k'^2
    ksqr = (e2 - e3) / (e1 - e3)
    ksqrp = (e1 - e2) / (e1 - e3)


    # Obliczenie omega1 i omega3 przy użyciu mpmath
    omega1 = (1 / sqrt(e1 - e3)) * ellipk(ksqr)
    omega3 = (1 / sqrt(e3 - e1)) * ellipk( ksqrp)

    # Obliczenie Im(tau)
    Imtau = (omega3 / omega1).imag

    # Warunek wyboru wartości omega1
    res = omega1 if Imtau > 0 else -omega1

    return res

In [60]:
def Omega3(g2,g3):
    # Współczynniki wielomianu 4*x**3 - g2*x - g3 = 0
    coeffs = [4, 0, -g2, -g3]

    # Znajdowanie pierwiastków numerycznych
    roots = np.roots(coeffs)
    
    # Sortowanie pierwiastków tak jak w SymPy
    roots = np.sort_complex(roots)
    e1, e2, e3 = roots

    # Obliczenie parametru k'^2
    ksqrp = (e1-e2)/(e1-e3)

    # Obliczenie omega3 przy użyciu mpmath
    omega3= (1/sqrt(e3-e1))*ellipk(ksqrp)

    return omega3

In [61]:
def Omega2(g2,g3):
    res = -Omega1(g2,g3) - Omega3(g2,g3)
    return res

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">$\wp$</h2>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">

Now using the encoded half-periods and the Jacobi thet encoded in the `mpmath` package, one can present a code for the Weierstrass elliptic function
</div>

In [128]:
def WeierstrassP(z,g2,g3):
    q = qfrom(tau=Omega3(g2,g3)/Omega1(g2,g3))
    FirstCase = (pi/(2*Omega1(g2,g3))) *jtheta(3, 0,q) *jtheta(4, 0, q) * jtheta(2, pi*z/(2*Omega1(g2,g3)),q)/jtheta(1, pi*z/(2*Omega1(g2,g3)),q)
    SecondCase = (1/12)*(pi/Omega1(g2,g3))**2 * (jtheta(2, 0,q)**4 + 2*jtheta(4, 0,q)**4)
    res = FirstCase**2 + SecondCase
    return res

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">$\wp^{-1}$</h2>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">

Based on formulas <a href="https://dlmf.nist.gov/23.6" target="_blank">(23.6.30)</a> for $t=\wp(z)$ we can write
$$
    \wp^{-1}(t)=z = \frac{1}{2} \int^\infty_t \frac{du}{\sqrt{(u-e_1)(u-e_2)(u-e_3)}}.
$$
Now we can use a definition of  <a href="https://en.wikipedia.org/wiki/Carlson_symmetric_form" target="_blank">Carlos elliptic integrals</a>:
$$
    R_F (x,y,z) = \frac{1}{2} \int^\infty_0 \frac{ds}{(s+x)(s+y)(s+z)}
$$
and by making a substitutuin $s+t=u$ notice that
$$
    \wp^{-1}(t) = R_F (z-e_1,z-e_2,z-e_3)
$$

</div>

In [129]:
def InverseWeierstrassP(z,g2,g3):
    # Współczynniki wielomianu 4*x**3 - g2*x - g3 = 0
    coeffs = [4, 0, -g2, -g3]

    # Znajdowanie pierwiastków numerycznych
    roots = np.roots(coeffs)
    
    # Sortowanie pierwiastków tak jak w SymPy
    roots = np.sort_complex(roots)
    e1, e2, e3 = roots
    
    return elliprf(z-e1, z-e2, z-e3)

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">$\wp'$</h2>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">

Now using formula provided by  <a href="https://functions.wolfram.com/EllipticFunctions/WeierstrassPPrime/27/02/03/0001/" target="_blank">Wolfram</a>
$$
    \wp'(z) = - \frac{\pi^3}{4\omega_1^3} \frac{\theta_2 \left( \frac{\pi z}{2\omega_1},q \right)\theta_3 \left( \frac{\pi z}{2\omega_1},q \right)\theta_4 \left( \frac{\pi z}{2\omega_1},q \right)\theta'_1 \left( 0,q \right)^3}{\theta_2 \left( 0,q \right)\theta_3 \left( 0,q \right)\theta_4 \left( 0,q \right)\theta_1 \left( \frac{\pi z}{2\omega_1},q \right)^3}
$$

</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 32px;">$\sigma$-function</h1>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">

As in previous section based on formulas <a href="https://dlmf.nist.gov/23.6" target="_blank">(23.6.8-9)</a> we can write
$$
    \sigma (z) = 2 \omega_1 \exp{\left( \frac{\eta_1 z^2}{2 \omega_1} \right)} \frac{\theta_1(\pi z/(2 \omega_1),q)}{\pi \theta '_1 (0,q)},
$$
where
$$
    \eta_1 = - \frac{\pi^2}{12 \omega_1} \frac{\theta_1 ''' (0,q)}{\theta_1'(0,q)}
$$

</div>

In [124]:
def eta1(g2,g3):
    q = qfrom(tau=Omega3(g2,g3)/Omega1(g2,g3))
    res = - (pi**2/(12*Omega1(g2,g3))) * (jtheta(1, 0, q, derivative=3)/jtheta(1, 0, q, derivative=1))
    return res

In [125]:
def WeierstrassSigma(z,g2,g3):
    q = qfrom(tau=Omega3(g2,g3)/Omega1(g2,g3))
    Exp = exp((eta1(g2, g3) * z**2) / (2 * Omega1(g2, g3)))
    res = 2*Omega1(g2,g3)*Exp * (jtheta(1, pi * z/(2*Omega1(g2,g3)),q)/(pi*jtheta(1, 0, q, derivative=1)))
    return res

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 32px;">$\zeta$-function</h1>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">

Here also based on basic property of $\zeta$ function <a href="https://dlmf.nist.gov/23.2" target="_blank">(23.2.8)</a> we can write
$$
    \zeta (z) = \frac{\sigma'(z)}{\sigma(z)} = \frac{\eta_1}{\omega_1} z + \frac{\theta'_1(\pi z/(2 \omega_1),q)}{\theta_1(\pi z/(2 \omega_1),q)} .
$$
</div>

In [114]:
def WeierstrassZeta(z,g2,g3):
    q = qfrom(tau=Omega3(g2,g3)/Omega1(g2,g3))
    firstP = eta1(g2,g3) * z/Omega1(g2,g3)
    secondP =  pi/(2*Omega1(g2,g3)) *jtheta(1,  pi * z/(2*Omega1(g2,g3)), q, derivative=1)/jtheta(1,  pi * z/(2*Omega1(g2,g3)), q)
    return firstP + secondP

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 32px;">Tests</h1>
</div>

wp=WeierstrassP(1.2321, 1.234, -4.213)
InverseWeierstrassP(wp, 1.234, -4.213)h

In [121]:
WeierstrassP(1.2321, 1.234, -4.213)

(0.4106033346576670977448281 - 1.060623335761549486890412e-16j)

In [127]:
wp=WeierstrassP(1.2321, 1.234, -4.213)
InverseWeierstrassP(wp, 1.234, -4.213)

(1.232100000000000037455484 + 5.314276203772776641693722e-17j)